In [1]:
from pyspark.sql import functions as F
from pyspark import SparkConf, StorageLevel
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.types import FloatType
import pandas as pd
import numpy as np

In [12]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf().setMaster('local[*]').setAppName("CarAccidents") \
.set("spark.driver.memory", "6g") \
.set("spark.executor.memory", "1g") \
.set("spark.driver.memoryOverhead", "256m") \
.set("spark.executor.memoryOverhead", "256m") \
.set("spark.sql.files.maxPartitionBytes", "64m") \
.set("spark.sql.execution.arrow.pyspark.enabled", "false")

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext


In [13]:
df = spark.read.options(header=True, escape='"', multiline=True,
                          ).parquet('hdfs://localhost:9000/converted_numbers/')

In [14]:
df_basic = df.select(["Severity", "State", "Description"])

In [15]:
df_pandas = df_basic.toPandas()

fifty_percent = int(len(df_pandas) * 0.5)
df_basic_50 = pd.DataFrame()
df_basic_50 = pd.concat([df_basic_50, df_pandas.iloc[0:fifty_percent]], ignore_index=True)
table = pd.crosstab(df_basic_50["State"], df_basic_50["Severity"])
print(table)

Severity     1       2       3     4
State                               
AL         325   37309   10529   893
AR          20    9005     301  1201
AZ        3601   66972    8182  2450
CA        4960  692053  131054  6195
CO         468   27239   12732  3368
CT          39   24859    8118  2068
DC          66    7710     509   367
DE         122    5659     359   787
FL        3290  365076   50804  6285
GA         513   45543   29954  6193
IA          23    8247    3456   960
ID           1    4690     107   283
IL         781   51879   28396  2097
IN         166   21978    7618  2692
KS          22    6723    1952   301
KY         133    9018    6149   559
LA        1201   62292    8555   903
MA         731   19563    8818   270
MD         193   41559    9162  4454
ME           1    1085     174    57
MI         548   56114   20128  3717
MN         231   77033   14635   412
MO         116   22721   12301  1246
MS          16    5234    1695   405
MT           0   13030      75   206
N

In [18]:
spark_df = spark.createDataFrame(df_basic_50)

spark_df.repartition(7).write.mode('overwrite').parquet('hdfs://localhost:9000/basic_50/')

AttributeError: 'DataFrame' object has no attribute 'iteritems'